In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from sympy import * # import all function names, whole library
from sympy.physics.vector import init_vprinting, dynamicsymbols, vpprint # for time non-static vector symbolic representation

init_vprinting() # Enhances display of dynamic symbols more beautifully, for normal ones uses init_printing()

# Symbolic representations of cart mass, point mass, length of rod, accn due to gravity, and force
m1, m2, l, g, F= symbols('m1 m2 l g F')

# Symbolic representation of state vectors x, x_dot, theta, theta_dot using dynamic symbols
x=dynamicsymbols('x')
x_dot= x.diff()
x_ddot= x.diff().diff()

theta=dynamicsymbols('θ')
theta_dot=theta.diff()
theta_ddot=theta.diff().diff()

# Writing down equations of motion of inverted pendulum
e1= (m1+m2)*x_ddot - m2*l*cos(theta)*theta_ddot + m2*l*sin(theta)*theta_dot**2 - F
e2= -cos(theta)*x_ddot + l*theta_ddot -g*sin(theta)

# We need to solve e1 and e2 to find x_ddot and theta_ddot since there are only two state variables each
# obtained from looking at the highest order of the eqations
result= solve([e1,e2],x_ddot,theta_ddot, dict=True)

# The obtained solutions were very long and complex, which can be simplified by using:
x_ddot_solved= simplify(result[0][x_ddot]) # result part is Accessing values from dictionary
theta_ddot_solved= simplify(result[0][theta_ddot]) 


### Generating state space equation i.e. xdot=Ax+Bu ### And write them in matrix form

stateFxn= Matrix([[x_dot],
                  [x_ddot_solved], 
                  [theta_dot], 
                  [theta_ddot_solved]])
                  # basically xdot part of state equation

# Computing the jacobians
Jstate= stateFxn.jacobian(x)
Jinput= stateFxn.jacobian(F)



In [ ]:
from multiprocessing.spawn import old_main_modules
import numpy as np
import matplotlib.pyplot as plt
import control as ct
from scipy.integrate import odeint
from sympy import * # import all function names, whole library
from sympy.physics.vector import init_vprinting, dynamicsymbols, vpprint # for time non-static vector symbolic representation

init_vprinting() # Enhances display of dynamic symbols more beautifully, for normal ones uses init_printing()

# Symbolic representations of cart mass, point mass, length of rod, accn due to gravity, and force
m1, m2, l, g, F= symbols('m1 m2 l g F')

# Symbolic representation of state vectors x, x_dot, theta, theta_dot using dynamic symbols
x=dynamicsymbols('x')
x_dot= x.diff()
x_ddot= x.diff().diff()

theta=dynamicsymbols('θ')
theta_dot=theta.diff()
theta_ddot=theta.diff().diff()

# Writing down equations of motion of inverted pendulum
e1= (m1+m2)*x_ddot - m2*l*cos(theta)*theta_ddot + m2*l*sin(theta)*theta_dot**2 - F
e2= -cos(theta)*x_ddot + l*theta_ddot -g*sin(theta)

# We need to solve e1 and e2 to find x_ddot and theta_ddot since there are only two state variables each
# obtained from looking at the highest order of the eqations
result= solve([e1,e2],x_ddot,theta_ddot, dict=True)

# The obtained solutions were very long and complex, which can be simplified by using:
x_ddot_solved= simplify(result[0][x_ddot]) # result part is Accessing values from dictionary
theta_ddot_solved= simplify(result[0][theta_ddot]) 


### Generating state space equation i.e. xdot=Ax+Bu ### And write them in matrix form
states= Matrix([[x],
                  [x_dot], 
                  [theta], 
                  [theta_dot]])
                  # basically x part of state equation

stateFxn= Matrix([[x_dot],
                  [x_ddot_solved], 
                  [theta_dot], 
                  [theta_ddot_solved]])
                  # basically xdot part of state equation

# Computing the jacobians
Jstate= stateFxn.jacobian(states) # jacobian of A
Jinput= stateFxn.jacobian([F])  # jacobian of B

# Adding constants to a dictionary to substitute them
consts={
    l: 1,
    g: 9.81,
    m1: 10,
    m2: 1
    }

# Substituting Constants into the jacobians
JstateSub= Jstate.subs(consts)
JinputSub= Jinput.subs(consts)

# Converting jacobians from symbolic expression to numerical
AmatFxn= lambdify([x, x_dot, theta, theta_dot, F],JstateSub)
BmatFxn= lambdify([x, x_dot, theta, theta_dot, F],JinputSub)

# Now providing equilibrium values to A and B matrices in order they were lambdified
# i.e. x, x_dot, theta ,theta_dot, F; they can be modified
x1, x2, x3 , x4, u= 0, 0, np.pi, 0, 0

A= AmatFxn(x1, x2, x3 , x4, u)
B= BmatFxn(x1, x2, x3 , x4, u)

# Creating output state matrices, assuming only angular components can be measured
C= np.array([
            [0,0,1,0],
            [0,0,0,1]
            ])

D= np.zeros((2,1))

# Defining a state space model
sysStateSpace= ct.ss(A,B,C,D)
print(sysStateSpace)

### Simulating open loop step response

startTime=0
endTime=20
samplesNum=1000
timeVector= np.linspace(startTime,endTime,samplesNum)
initState= np.array([0,0,np.pi/3,0])
controlInput= np.ones(samplesNum)

OLsimulation=ct.forced_response(sysStateSpace,timeVector,controlInput,initState) # cannot be used for non-linearized system, if so then have to use odeint with a function that can return derivatives


"""
    These are the structure of outputs otained from forced response function
    
OLsimulation.time
OLsimulation.outputs
OLsimulation.states
OLsimulation.inputs
"""

# Create a figure with 4 subplots (2 rows, 2 column)
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 12))

# Plotting Cart and pendulum states
ax1.plot(OLsimulation.time, OLsimulation.states[0,:], 'b', linewidth=4, label='x')
ax1.set_xlabel('time', fontsize=16)
ax1.set_ylabel('Cart position', fontsize=16)
ax1.legend(fontsize=14)
ax1.tick_params(axis='both', which='major', labelsize=14)
ax1.grid(True)

ax2.plot(OLsimulation.time, OLsimulation.states[1,:], 'b', linewidth=4, label='x')
ax2.set_xlabel('time', fontsize=16)
ax2.set_ylabel('Cart velocity', fontsize=16)
ax2.legend(fontsize=14)
ax2.tick_params(axis='both', which='major', labelsize=14)
ax2.grid(True)

ax3.plot(OLsimulation.time, OLsimulation.states[2,:], 'b', linewidth=4, label='x')
ax3.set_xlabel('time', fontsize=16)
ax3.set_ylabel('Pendulum Position', fontsize=16)
ax3.legend(fontsize=14)
ax3.tick_params(axis='both', which='major', labelsize=14)
ax3.grid(True)

ax4.plot(OLsimulation.time, OLsimulation.states[3,:], 'b', linewidth=4, label='x')
ax4.set_xlabel('time', fontsize=16)
ax4.set_ylabel('Pendulum velocity', fontsize=16)
ax4.legend(fontsize=14)
ax4.tick_params(axis='both', which='major', labelsize=14)
ax4.grid(True)

####      Designing LQR controller      #####

#defining desired states
xd= np.array([[0],[0],[np.pi],[0]])

#determining ud
udMat= np.matmul(np.matmul(np.linalg.inv(np.matmul(B.T,B)), B.T),A) # this is matrix multiplications consisting of transposition and inverses
                                                                    #   (((B^T)*B)^-1)*B^T)*A
ud= -np.matmul(udMat,xd)

reqdInput= ud[0,0]*np.ones(samplesNum)

CLsimulation=ct.forced_response(sysStateSpace,timeVector,reqdInput,initState) 

# Appending plots of Cart and pendulum states after providing required force
ax1.plot(CLsimulation.time, CLsimulation.states[0,:], 'r', linewidth=4, label='x')
ax1.set_xlabel('time', fontsize=16)
ax1.set_ylabel('Cart position', fontsize=16)
ax1.legend(fontsize=14)
ax1.tick_params(axis='both', which='major', labelsize=14)
ax1.grid(True)

ax2.plot(CLsimulation.time, CLsimulation.states[1,:], 'r', linewidth=4, label='x')
ax2.set_xlabel('time', fontsize=16)
ax2.set_ylabel('Cart velocity', fontsize=16)
ax2.legend(fontsize=14)
ax2.tick_params(axis='both', which='major', labelsize=14)
ax2.grid(True)

ax3.plot(CLsimulation.time, CLsimulation.states[2,:], 'r', linewidth=4, label='x')
ax3.set_xlabel('time', fontsize=16)
ax3.set_ylabel('Pendulum Position', fontsize=16)
ax3.legend(fontsize=14)
ax3.tick_params(axis='both', which='major', labelsize=14)
ax3.grid(True)

ax4.plot(CLsimulation.time, CLsimulation.states[3,:], 'r', linewidth=4, label='x')
ax4.set_xlabel('time', fontsize=16)
ax4.set_ylabel('Pendulum velocity', fontsize=16)
ax4.legend(fontsize=14)
ax4.tick_params(axis='both', which='major', labelsize=14)
ax4.grid(True)


### Selecting Q and R values and computing using LQR
Q= 100*np.eye(4)
R= 1*np.array([[1]])

K, S, E = ct.lqr(sysStateSpace, Q, R) # where, K= gain matrix, S= Riccati solution, E= eigenvalues

# defining new state space equation
Acl= A - np.matmul(B,k)
Bcl= -Acl

controlledSysStateSpace= ct.ss(Acl,Bcl,C,D)
timeVector= np.linspace(startTime,endTime,samplesNum)







In [ ]:

# Now choosing values for masses, gravity, and length and then substituting values of known quantities in solved equations
##### We can change these constants with desired values #####
lval=1
m1val=10
m2val=1
gval=9.81

x_ddot_subbed= x_ddot_solved.subs(l,lval).subs(g,gval).subs(m1,m1val).subs(m2,m2val)
theta_ddot_subbed= theta_ddot_solved.subs(l,lval).subs(g,gval).subs(m1,m1val).subs(m2,m2val)
# Converting symbolic expressions into python/mathematical functions using lambdify from sympy
func_x_ddot= lambdify([x, x_dot, theta, theta_dot, F], x_ddot_subbed) # Placing values/symbols for each state variables and input into equation x_ddot_solved
func_theta_ddot= lambdify([x, x_dot, theta, theta_dot, F], theta_ddot_subbed)

def stateSpaceModel(state, t, timePoints, forceArray):  # the state argument is provided automatically by ode solver which starts from initial conditions that we provide
    x, x_dot, theta, theta_dot = state  # unpacking current state variables. i.e. x=state[0], x_dot=state[1], and so on...
    
    forceApplied= np.interp(t, timePoints, forceArray)
    x_ddot= func_x_ddot(x, x_dot, theta, theta_dot, forceApplied) # Placing numerical values for each state variables and inputs into now converted mathematical function
    theta_ddot= func_theta_ddot(x, x_dot, theta, theta_dot, forceApplied)
    
    return [x_dot, x_ddot, theta_dot, theta_ddot]   # returning derivatives of state variables, i.e. d/dt(state)

# Defining simulation parameters #### Can change it
startTime=0
endTime=25
timeSteps=5000

simTime= np.linspace(startTime,endTime,timeSteps)   # Generating discrete time interval
# forceInput= np.sin(simTime)+np.cos(2*simTime) # Generating sine and cosine function over time
forceInput= np.zeros(shape=(simTime.shape)) # Zero force applied

#plt.plot(simTime,forceInput)

##  Simulating the dynamics of the system
initState= np.array([0,0,np.pi/3,0])   # initial conditions position, velocity, and angular velocity are zero, while angular displacement is pi/3 rad

solution= odeint(stateSpaceModel, initState, simTime, args=(simTime, forceInput))   # Extra arguments required for our function time points and force applied is also provided
# solution.shape ## it shows that there are 5000 rows(values due to timesteps), and 4 cols (state variables)

# save the simulation data
# the save file is opened by another Python script that is used to animate the trajectory
np.save('simulationData.npy', solution)


# Create a figure with 2 subplots (2 rows, 1 column)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

# First subplot: Cart states
ax1.plot(simTime, solution[:,0], 'b', linewidth=4, label='x')
ax1.plot(simTime, solution[:,1], 'r', linewidth=4, label='x_dot')
ax1.set_xlabel('time', fontsize=16)
ax1.set_ylabel('Cart states', fontsize=16)
ax1.legend(fontsize=14)
ax1.tick_params(axis='both', which='major', labelsize=14)
ax1.grid()

# Second subplot: Pendulum states
ax2.plot(simTime, solution[:,2], 'b', linewidth=4, label='θ')
ax2.plot(simTime, solution[:,3], 'r', linewidth=4, label='θ_dot')
ax2.set_xlabel('time', fontsize=16)
ax2.set_ylabel('Pendulum states', fontsize=16)
ax2.legend(fontsize=14)
ax2.tick_params(axis='both', which='major', labelsize=14)
ax2.grid()

# Adjust layout to prevent overlapping
plt.tight_layout()

# Save the figure
plt.savefig('all_states.png', dpi=600)
plt.show()